In [ ]:
# Chan's Convex Hull O(n log h) - Tom Switzer <thomas.switzer@gmail.com>

TURN_LEFT, TURN_RIGHT, TURN_NONE = (1, -1, 0)

def turn(p, q, r):
    """Returns -1, 0, 1 if p,q,r forms a right, straight, or left turn."""
    return orientation(p, q, r)

def _keep_left(hull, r):
    while len(hull) > 1 and turn(hull[-2], hull[-1], r) != TURN_LEFT:
            hull.pop()
    return (not len(hull) or hull[-1] != r) and hull.append(r) or hull

from functools import reduce
def _graham_scan(points):
    """Returns points on convex hull of an array of points in CCW order."""
    points.sort()
    lh = reduce(_keep_left, points, [])
    uh = reduce(_keep_left, reversed(points), [])
    return lh.extend(uh[i] for i in range(1, len(uh) - 1)) or lh

def _rtangent(hull, p):
    """Return the index of the point in hull that the right tangent line from p
    to hull touches.
    """
    l, r = 0, len(hull)
    l_prev = turn(p, hull[0], hull[-1])
    l_next = turn(p, hull[0], hull[(l + 1) % r])
    while l < r:
        c = (l + r) // 2
        c_prev = turn(p, hull[c], hull[(c - 1) % len(hull)])
        c_next = turn(p, hull[c], hull[(c + 1) % len(hull)])
        c_side = turn(p, hull[l], hull[c])
        if c_prev != TURN_RIGHT and c_next != TURN_RIGHT:
            return c
        elif c_side == TURN_LEFT and (l_next == TURN_RIGHT or
                                      l_prev == l_next) or \
                c_side == TURN_RIGHT and c_prev == TURN_RIGHT:
            r = c               # Tangent touches left chain
        else:
            l = c + 1           # Tangent touches right chain
            l_prev = -c_next    # Switch sides
            l_next = turn(p, hull[l], hull[(l + 1) % len(hull)])
    return l

def _min_hull_pt_pair(hulls):
    """Returns the hull, point index pair that is minimal."""
    h, p = 0, 0
    for i in range(len(hulls)):
        j = min(range(len(hulls[i])), key=lambda j: hulls[i][j])
        if hulls[i][j] < hulls[h][p]:
            h, p = i, j
    return (h, p)

def _next_hull_pt_pair(hulls, pair):
    """
    Returns the (hull, point) index pair of the next point in the convex
    hull.
    """
    p = hulls[pair[0]][pair[1]]
    next = (pair[0], (pair[1] + 1) % len(hulls[pair[0]]))
    for h in (i for i in range(len(hulls)) if i != pair[0]):
        s = _rtangent(hulls[h], p)
        q, r = hulls[next[0]][next[1]], hulls[h][s]
        t = turn(p, q, r)
        if t == TURN_RIGHT or t == TURN_NONE and distance(p, r) > distance(p, q):
            next = (h, s)
    return next

def convex_hull(pts):
    """Returns the points on the convex hull of pts in CCW order."""
    for m in (1 << (1 << t) for t in range(len(pts))):
        hulls = [_graham_scan(pts[i:i + m]) for i in range(0, len(pts), m)]
        hull = [_min_hull_pt_pair(hulls)]
        for throw_away in range(m):
            p = _next_hull_pt_pair(hulls, hull[-1])
            if p == hull[0]:
                return [hulls[h][i] for h, i in hull]
            hull.append(p)

In [ ]:
# TODO: could greatly benefit from keeping track of which hulls each point is in when choosing the next point

def partition_points(input_set: list[tuple[float, float]], num_partitions: int) -> list[list[tuple[float, float]]]:
    partition_size = math.ceil(len(input_set) / num_partitions)
    return [input_set[i:i + partition_size] for i in range(0, len(input_set), partition_size)]

def orientation(p: tuple[float, float], q: tuple[float, float], r: tuple[float, float]) -> int:
    # 0 -> colinear, 1 -> clockwise, -1 -> counterclockwise
    area = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
    if area == 0:
        return 0
    return 1 if area > 0 else -1

def find_tangent(p: tuple[float, float], hull: list[tuple[float, float]]) -> int:
    if len(hull) == 1:
        return 0
        
    n = len(hull)
    left, right = 0, n - 1

    left_before = orientation(p, hull[0], hull[-1])
    left_after = orientation(p, hull[0], hull[(left + 1) % n])

    while left < right:
        mid = (left + right) // 2

        mid_before = orientation(p, hull[mid], hull[abs(mid - 1) % n])
        mid_after = orientation(p, hull[mid], hull[(mid + 1) % n])
        mid_side = orientation(p, hull[left], hull[mid])
 
        if mid_before != -1 and mid_after != -1:
            return mid
        elif (mid_side == 1 and (left_after == -1 or left_before == left_after)) \
                or (mid_side == -1 and mid_before == -1):
            right = mid
        else:
            left = mid + 1
            left_before = -mid_after
            left_after = orientation(p, hull[left], hull[(left + 1) % n])
        
    return left

TURN_LEFT, TURN_RIGHT, TURN_NONE = (1, -1, 0)
def _rtangent_(p, hull):
    """Return the index of the point in hull that the right tangent line from p
    to hull touches.
    """
    if len(hull) == 1:
        return 0
    
    l, r = 0, len(hull)
    l_prev = orientation(p, hull[0], hull[-1])
    l_next = orientation(p, hull[0], hull[(l + 1) % r])
    while l < r:
        c = (l + r) // 2
        c_prev = orientation(p, hull[c], hull[abs(c - 1) % len(hull)])
        c_next = orientation(p, hull[c], hull[(c + 1) % len(hull)])
        c_side = orientation(p, hull[l], hull[c])
        if c_prev != TURN_RIGHT and c_next != TURN_RIGHT:
            return c
        elif c_side == TURN_LEFT and (l_next == TURN_RIGHT or
                                      l_prev == l_next) or \
                c_side == TURN_RIGHT and c_prev == TURN_RIGHT:
            r = c               # Tangent touches left chain
        else:
            l = c + 1           # Tangent touches right chain
            l_prev = -c_next    # Switch sides
            l_next = orientation(p, hull[l], hull[(l + 1) % len(hull)])
    return l

def visualize_convex_hulls(
        partitions: list[tuple[float, float]],
        hulls: list[list[tuple[float, float]]], 
    ) -> None:

    for i in range(len(hulls)):
        x = [i[0] for i in partitions[i]]
        y = [i[1] for i in partitions[i]]
        plt.scatter(x, y)
        x = [i[0] for i in hulls[i]]
        y = [i[1] for i in hulls[i]]
        # close the loop
        x.append(hulls[i][0][0])
        y.append(hulls[i][0][1])
        plt.plot(x, y)
        
    plt.show()

def get_hull_leftmost_point(hulls: list[list[tuple[float, float]]]) -> tuple[int, int]:
    # returns the index of the hull and the index of the leftmost point in the hull
    min_x = math.inf
    min_x_index = 0
    min_x_hull_index = 0
    for i in range(len(hulls)):
        for j in range(len(hulls[i])):
            if hulls[i][j][0] < min_x:
                min_x = hulls[i][j][0]
                min_x_index = j
                min_x_hull_index = i
    return min_x_hull_index, min_x_index


def _min_hull_pt_pair(hulls):
    """Returns the hull, point index pair that is minimal."""
    h, p = 0, 0
    for i in range(len(hulls)):
        j = min(range(len(hulls[i])), key=lambda j: hulls[i][j])
        if hulls[i][j] < hulls[h][p]:
            h, p = i, j
    return (h, p)

def turn(p, q, r):
    """Returns -1, 0, 1 if p,q,r forms a right, straight, or left turn."""
    return orientation(p, q, r)

def rtangent(hull, p):
    """Return the index of the point in hull that the right tangent line from p
    to hull touches.
    """
    l, r = 0, len(hull)
    l_prev = turn(p, hull[0], hull[-1])
    l_next = turn(p, hull[0], hull[(l + 1) % r])
    while l < r:
        c = (l + r) // 2
        c_prev = turn(p, hull[c], hull[abs(c - 1) % len(hull)])
        c_next = turn(p, hull[c], hull[(c + 1) % len(hull)])
        c_side = turn(p, hull[l], hull[c])
        if c_prev != TURN_RIGHT and c_next != TURN_RIGHT:
            return c
        elif c_side == TURN_LEFT and (l_next == TURN_RIGHT or
                                      l_prev == l_next) or \
                c_side == TURN_RIGHT and c_prev == TURN_RIGHT:
            r = c               # Tangent touches left chain
        else:
            l = c + 1           # Tangent touches right chain
            l_prev = -c_next    # Switch sides
            l_next = turn(p, hull[l], hull[(l + 1) % len(hull)])
    return l

def get_next_hull_and_point(
        hulls: list[list[tuple[float, float]]], 
        prev_hull_i: int, 
        prev_point_i: int
    ) -> tuple[int, int]:
    
    prev_point = hulls[prev_hull_i][prev_point_i]
    next_point = (prev_hull_i, (prev_point_i - 1) % len(hulls[prev_hull_i]))
    for h_i in range(len(hulls)):
        if h_i == prev_hull_i:
            continue
        rtan_i = rtangent(hulls[h_i], prev_point)
        end_point = hulls[next_point[0]][next_point[1]]
        rtan_point = hulls[h_i][rtan_i]
        turn_type = turn(prev_point, end_point, rtan_point)
        if turn_type == TURN_RIGHT or (turn_type == TURN_NONE and distance(prev_point, rtan_point) > distance(prev_point, end_point)):
            next_point = (h_i, rtan_i)

    return next_point

def chan(input_set: list[tuple[float, float]]) -> list[tuple[float, float]] | None:
    """
    Returns the list of points that lie on the convex hull (chan's algorithm)
            Parameters:
                    input_set (list): a list of 2D points

            Returns:
                    output_set (list): a list of 2D points
    """
    n = len(input_set)

    if n < 3:
        return input_set
    
    m = 3
    while m < n:
        hulls = [graham_scan(input_set[i:i + m]) for i in range(0, n, m)]
        convex_hull = [_min_hull_pt_pair(hulls)]
        for _ in range(m):
            last_hull_i, last_point_i = convex_hull[-1]
            next_point = get_next_hull_and_point(hulls, last_hull_i, last_point_i)
            if next_point == convex_hull[0]:
                return [hulls[h_i][p_i] for h_i, p_i in convex_hull]
            convex_hull.append(next_point)
        m = min(m * m, n)